In [1]:
import cntk
import numpy
import cv2
from matplotlib import pyplot as plt
from PIL import Image
import time
import os
import cntk as C

C:\Users\Mig\Anaconda3\lib\site-packages\cntk\cntk_py_init.py:84: UserWarning: 

################################################ Missing optional dependency (GPU-Specific) ################################################
   CNTK may crash if the component that depends on those dependencies is loaded.
   Visit https://docs.microsoft.com/en-us/cognitive-toolkit/Setup-Windows-Python#optional-gpu-specific-packages for more information.
############################################################################################################################################
If you intend to use CNTK without GPU support, you can ignore the (likely) GPU-specific warning!
############################################################################################################################################

  warnings.warn(WARNING_MSG_GPU_ONLY % ('GPU-Specific', 'https://docs.microsoft.com/en-us/cognitive-toolkit/Setup-Windows-Python#optional-gpu-specific-packages'))


In [2]:
def create_reader(path, is_training, input_dim, num_label_classes):
    featureStream = cntk.io.StreamDef(field='features', shape=input_dim, is_sparse=False)
    deserailizer = cntk.io.CTFDeserializer(path, cntk.io.StreamDefs(features=featureStream))
    return cntk.io.MinibatchSource(deserailizer, randomize=is_training, max_sweeps=1, randomization_window_in_chunks=0)

def load_model(path):
    model = cntk.ops.functions.load_model(path)
    return model

def predict(model, data):
    return model.eval(data)

def do_predict(model_path, data_path, len_data):
    model = load_model(model_path)
    reader = create_reader(data_path, False, 3, 3)
    input_map = {
        input: reader.streams.features
    }
    all_predict_value = numpy.zeros((1,3), dtype=int)
    
    #num_minibatches = len_data//100
    #print(len_data, num_minibatches)
    for i in range(len_data):
        data = reader.next_minibatch(1, input_map=input_map)
        feature_data = data[input].asarray()
        predict_value = predict(model, feature_data)
        all_predict_value = numpy.append(all_predict_value, predict_value, axis=0)
        
    print("Finish Prediction")
    return all_predict_value 

def img_to_txt(path):
    txt_name = "image_rgb_file2.txt"
    img = Image.open(path)
    pixel_list = []
    for i in range(img.size[0]):    
        for j in range(img.size[1]):
            pixel_list.append(img.getpixel((i,j)))
            
    pixel_set = set(pixel_list)
    count = 1
    with open(txt_name, 'w') as outfile:
        for p in pixel_set:
            outfile.write("|no " + str(count) + "|features " + str(p[0]) + " " + str(p[1]) + " " + str(p[2]) + "\n")
            count+=1
    print("Finish Convert Image")
    
    return pixel_set, txt_name ,len(pixel_set)

def data_to_img(predict_value, pixel_set, image_path, img_name):
    i = 0
    combined_dict = {}
    for e in pixel_set:
        #print(e, predict_value[i])
        combined_dict[e] = predict_value[i]
        i+=1
    
    img = Image.open(image_path)
    pixels = img.load()
    
    for i in range(img.size[0]):    
        for j in range(img.size[1]):
            key = img.getpixel((i,j))
            #print(img.getpixel((i,j)), "=>" ,combined_dict[key])
            R = int(combined_dict[key][0])
            G = int(combined_dict[key][1])
            B = int(combined_dict[key][2])
            
            R = correct_range(R)
            G = correct_range(G)
            B = correct_range(B)
            
            pixels[i,j] = (R, G, B)
            
    img.save(img_name)
    print("Finish Write Image")
    
def correct_range(pixel_value):
    if pixel_value < 0:
        pixel_value = 0
        
    if pixel_value > 255:
        pixel_value = 255
        
    return pixel_value
    
def create_img(model_path, image_path, output_image_name):
    pixel_set, pixel_txt_path, len_data = img_to_txt(image_path)
    predict_value = do_predict(model_path, pixel_txt_path, len_data)
    predict_value = numpy.delete(predict_value, 0, 0)
    predict_value = predict_value.round()
    data_to_img(predict_value, pixel_set, image_path, output_image_name)


images_dir = "./Images/"
output_dir = "./Images_Predict/"

#model_path = "./Model_New/2_iPadtoMotoCModel.model" 
model_path = "./iPadtoMotoC[Avg].model"
count = 0
sum_augment_time = 0

'''
isGPU = C.device.try_set_default_device(C.device.gpu(1))
if isGPU:
    print("Using GPU 1")
else:
    print("Using CPU")
'''

for filename in os.listdir(images_dir):
    if filename.endswith(".jpg") and "iPad" in filename:
        image_path = images_dir + filename
        output_image_name = output_dir + filename
        start_time = time.time()
        create_img(model_path, image_path, output_image_name)
        print("Finish : ", filename)
        aug_time = time.time() - start_time
        print("--- %s seconds ---" % (aug_time))
        sum_augment_time += aug_time
        count += 1
        
print("Finish Augment " + str(count) + " Images in " + str(sum_augment_time))
#with open('Finish2.txt', 'w') as finishFile:
    #finishFile.write("Finish Augment " + str(count) + " Images in " + str(sum_augment_time))

Finish Convert Image
Finish Prediction
Finish Write Image
Finish :  Arm(iPad)_resize.jpg
--- 20.810089588165283 seconds ---
Finish Convert Image
Finish Prediction
Finish Write Image
Finish :  ATM(iPad)_resize.jpg
--- 264.2399756908417 seconds ---
Finish Convert Image
Finish Prediction
Finish Write Image
Finish :  Book(iPad)_resize.jpg
--- 688.6650156974792 seconds ---
Finish Convert Image
Finish Prediction
Finish Write Image
Finish :  Bucket(iPad)_resize.jpg
--- 160.82775020599365 seconds ---
Finish Convert Image
Finish Prediction
Finish Write Image
Finish :  Field(iPad)_resize.jpg
--- 85.37635540962219 seconds ---
Finish Convert Image
Finish Prediction
Finish Write Image
Finish :  Opposite(iPad)_resize.jpg
--- 14.110146760940552 seconds ---
Finish Convert Image
Finish Prediction
Finish Write Image
Finish :  Shirt(iPad)_resize.jpg
--- 86.07389569282532 seconds ---
Finish Convert Image
Finish Prediction
Finish Write Image
Finish :  Sky(iPad)_resize.jpg
--- 55.00462508201599 seconds ---
